In [20]:
import requests
import json
from datetime import datetime, timedelta

# Create timestamps
now = datetime.now()
first_event_time = now - timedelta(minutes=5)  # 5 minutes ago
last_event_time = now

# Build test incident payload
test_incident = {
    "id": f"TEST_{int(now.timestamp())}",
    "timestamp": now.isoformat(),
    "description": json.dumps({
        "rule_id": "834613",
        "rule_description": "1&1 security alert",
        "rule_level": 10,
        "total_events": 2,
        "agent_names": ["Blue Iris"],
        "first_event_timestamp": first_event_time.isoformat(),
        "last_event_timestamp": last_event_time.isoformat(),
        "sample_event": {
            "rule": {
                "id": "834613",
                "description": "Critical Camera Alert",
                "level": 10
            },
            "agent": {
                "name": "Blue Iris",
                "id": "001"
            },
            "timestamp": first_event_time.isoformat(),
            "data": {
                "service": "nginx",
                "status": "stopped",
                "exit_code": 1
            }
        },
        "csv_path": "/incidents/test/events.csv"
    }),
    "acknowledged": False,
    "escalated": False
}

# Send POST request using requests module
response = requests.post(
    url='http://localhost:8000/incidents/',
    headers={'Content-Type': 'application/json'},
    json=test_incident
)

if response.status_code != 200:
    print('Error:', response.status_code, response.text)
else:
    print('Response:', response.json())

Response: {'message': 'Incident created', 'id': 'TEST_1731604758'}


In [21]:
from sqlalchemy import Column, Integer, String, Boolean, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from passlib.context import CryptContext
import os

# Database setup
DATABASE_URL = os.getenv("DATABASE_URL", "sqlite:///./users.db")
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# Password hashing
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True, index=True)
    username = Column(String, unique=True, index=True)
    hashed_password = Column(String)
    is_active = Column(Boolean, default=True)

    @property
    def password(self):
        raise AttributeError("password is not a readable attribute")

    @password.setter
    def password(self, password):
        self.hashed_password = pwd_context.hash(password)

def create_tables():
    Base.metadata.create_all(bind=engine)

def create_default_admin():
    db = SessionLocal()
    admin = db.query(User).filter(User.username == "admin").first()
    if not admin:
        admin = User(username="admin")
        admin.password = "admin"
        db.add(admin)
        db.commit()
    db.close()

def verify_auth(username: str, password: str):
    db = SessionLocal()
    user = db.query(User).filter(User.username == username).first()
    db.close()
    
    if not user:
        return False
    
    return pwd_context.verify(password, user.hashed_password)

# Initialize database and create admin user
create_tables()
create_default_admin()